## Python imports

In [1]:
import requests
import json
import time
from jinja2 import Template
import os
import sys
from random import randint
import urllib3
urllib3.disable_warnings()

### _Load Django into Jupyter_

In [2]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

In [3]:
with open("/templates/splunk.py", "r+") as file:
    data = file.read()
    if not os.path.exists("/bookstore/bookstore"):
        os.makedirs("/bookstore/bookstore")
    with open("/bookstore/bookstore/splunk.py", "w") as file:
        write_file = file.write(data)

# print(open("/bookstore/bookstore/splunk.py").read())

In [4]:
file = '/bookstore/bookstore/models.py'    

splunk = True
# splunk = False

with open(file, 'w') as file:
    template = Template(open('/templates/models.j2', 'r').read())
    print(template.render(splunk=splunk))
    file.write(template.render(splunk=splunk))
    file.close()

from django.db import models
from django.conf import settings
from django.contrib.auth.models import User
from django.db.models.signals import post_save
from django.dispatch import receiver
import uuid
from simple_history.models import HistoricalRecords
from .splunk import *



from django.db import models
from simple_history.models import HistoricalRecords

class AbstractModel(models.Model):
    id = models.UUIDField(primary_key=True, default=uuid.uuid4, editable=False)
    #question = models.CharField(max_length=200)
    #pub_date = models.DateTimeField('date published')
    changed_by = models.ForeignKey('auth.User', on_delete=models.PROTECT, editable=False)
    history = HistoricalRecords(inherit=True)

    @property
    def _history_user(self):
        return self.changed_by

    @_history_user.setter
    def _history_user(self, value):
        self.changed_by = value

    class Meta:
        abstract = True

class Author(AbstractModel):
    name = models.CharField("Author", max_l

## Challenge #8
### Logging to Splunk
Create a token and post a log message

In [5]:
# def get_token(url, username, password):
#     payload = {'username': username, 'password': password}
#     headers = {'Content-type': 'application/json'}
#     return requests.post(f'{url}/token/', json=payload, headers=headers).json()['access']

# url = 'http://bookstore:8000/api'
# token = get_token(url, 'carlos.carrot', 'secret123')
# headers = {'Content-type': 'application/json', 'Authorization': f'Bearer {token}'}
# response = requests.get(f'{url}/book', headers=headers).json()
# books = response['results']
# while response['count'] > len(books):
#     books += requests.get(response['next'], headers=headers).json()['results']

# rand_book = books[randint(0, len(books)-1)]
# rand_book = requests.get(rand_book['url'], headers=headers).json()
# rand_book['title'] += " Quirk"
# rand_book
# book = requests.put(rand_book['url'], json=rand_book, headers=headers).json()
# book

In [6]:
# from bookstore.models import *

# my_book = Book.objects.get(id=book['id'])

# for h in my_book.history.all():
#     print(h)

# my_book.history.all()

In [7]:
# requests.get(f'{url}/book?author=Ray%20Bradbury', headers=headers).json()

In [8]:
from bookstore.splunk import *

app = 'bookstore'

index = get_splunk_index(app)
print(json.dumps(index, indent=4, sort_keys=True))

input_http, splunk_token = get_splunk_input_http(app)
print(json.dumps(input_http, indent=4, sort_keys=True))

{
    "id": "https://splunk:8089/servicesNS/nobody/search/data/indexes/bookstore_index",
    "name": "bookstore_index",
    "updated": "2022-08-25T09:56:24+00:00"
}

{
    "description": "Bookstore Token",
    "id": "https://splunk:8089/servicesNS/nobody/splunk_httpinput/data/inputs/http/http%3A%252F%252Fbookstore",
    "name": "http://bookstore",
    "token": "650ddb57-2112-479c-bbf0-da56b8ba1717",
    "updated": "2022-08-25T09:56:26+00:00"
}


## Check Collector

In [9]:
url = "https://splunk:8088/services/collector"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Splunk {splunk_token}'
}

response = requests.get(f"{url}/health", headers=headers, verify=False).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "code": 17,
    "text": "HEC is healthy"
}


## Upload log entries

In [16]:
from bookstore.models import HistoricalBook

HistoricalBook.objects.last().changed_by.username
HistoricalBook.objects.all()

<QuerySet [<HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 10:02:41.270661+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:54:46.616683+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:54:02.851014+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:53:11.057830+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:51:43.084093+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:51:06.977031+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:50:33.743273+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:50:00.716673+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:49:33.971927+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:49:12.682485+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:47:34.061855+00:00>, <HistoricalBook: Pride and Prejudice - HHH as of 2022-08-25 09:36

In [ ]:
import random

for i in range(0,4):
    users = ["Anna", "Bob", "Carlos"]
    dict = { 'message' : "Dummy test python client log", 'user': random.choice(users)}
    log = splunklog('bookstore', dict)
    print(log)

In [ ]:
dict = { 'message' : "Dummy test Python client log", 'id': 12345}
log = splunklog('bookstore', dict)
print(log)

In [ ]:
from time import sleep

endpoint = "https://splunk:8089"
auth = ('admin', 'secret123')
data = {
    'search': f'search index={app}_index AND id=12345',
    'output_mode':'json'
}
response = requests.post(f"{endpoint}/services/search/jobs", data=data, auth=auth, verify=False).json()
sid=response['sid']

while True:
    response = requests.get(f"{endpoint}/services/search/jobs/{sid}", data=data, auth=auth, verify=False).json()
    if response['entry'][0]['content']['isDone']:
        break
    else:
        sleep(1)

url = f"{endpoint}{response['entry'][0]['links']['events']}"
response = requests.get(url, data=data, auth=auth, verify=False).json()
for log in response['results']:
    print(json.dumps(json.loads(log['_raw']), indent=4, sort_keys=True))